# CDS services in jupyter notebooks
Welcome to a Jupyter notebook demonstrating how to access some CDS services with Python. Jupyter notebooks offer an interactive way to use Python. If you are viewing this notebook with Binder you can execute every cell of code either by pushing the little 'play' <img src="images/jupyter_run-cell.png" alt="" style="width:70px; display: inline-block;"/> symbol on the left of the cell (appears when the mouse pointer moves in its vincitity) or by selecting a cell and pressing 'Shift' + 'Enter' at the same time. Note that cells that have not yet been executed have no number assigned to them, later on they are assigned the number of the order in which they were executed. If they are run again, they get a new number. 

The aim of this tutorial is to open a Aladin Lite widget, find and download a catalogue from Vizier and overlay the sources of this catalogue on the sky view in the Aladin Lite widget. 

Python draws its power from the large variety of packages that are available for everyone to install and use. For this tutorial we will be using two packages: `ipyaladin` (https://github.com/cds-astro/ipyaladin, allows to load Aladin Lite in a jupyter notebook) and `Vizier` from the `astroquery` package (https://astroquery.readthedocs.io/en/latest/index.html#introduction, allows to query VizieR from this notebook). Let's import these packages by running the following cell:

In [1]:
from astroquery.vizier import Vizier
import ipyaladin.aladin_widget as ipyal
import pyvo

print('Hello there, the packages are imported and we are ready to go :)')


Hello there, the packages are imported and we are ready to go :)


## Start Aladin Lite
We start this tutorial by initialising Aladin Lite. In the first line of code we tell Python that the variable `aladin` is now assigned to an Aladin Lite widget. This widget is supposed to **center** on the Antennae galaxies, set a **field of view** of 0.7deg and show colour images from the DSS **survey**. Typing `aladin` in the second line of code, tells the notebook to display the widget. 

In [2]:
aladin = ipyal.Aladin(target='Antennae', fov=0.7, survey='P/DSS2/color')
aladin


Aladin(fov=0.7, options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_survey', 'ov…

In [3]:
aladin.target = 'M101'


## Find and download tables from VizieR
Besides using Aladin we can also query VizieR from within this Jupyter notebook. To do so we use the `Vizier` subpackage from the `astroquery` package. As we remember from before, the catalogue of peculiar galaxies was called "Arp's peculiar galaxies" and was published by Webb. Hence, we ask `Vizier` to find all catalogues that have a match with the keywords 'Arp Webb' and write the result into the variable `catalog_list_arp` (first line of code). Then we tell Python to `print` out the query result in a readable way (second line of code).  

In [4]:
catalog_list_arp = Vizier.find_catalogs('Arp')
for k, v in catalog_list_arp.items():
    print(k, ': ', v.description)


VII/13 :  Globular-Cluster Catalog (Arp 1965)
VII/74A :  Atlas of Peculiar Galaxies (Arp 1966)
VII/170 :  Catalogue of Southern Peculiar Galaxies and Associations (Arp+, 1987)
VII/192 :  Arp's Peculiar Galaxies (Webb 1996)
J/ApJ/641/763 :  UBVI photometry for Arp 220 star clusters (Wilson+, 2006)
J/ApJ/660/167 :  Spitzer AGN candidates in CDFN (Donley+, 2007)
J/ApJ/799/10 :  Arp 220 6 and 33GHz images (Barcos-Munoz+, 2015)
J/A+A/319/33 :  Identification of X-ray sources around Seyferts (Arp 1997)
J/A+A/396/473 :  VI Photometry in M81 Group (Makarova+ 2002)
J/A+A/529/A104 :  Coordinates for Arp's Baade's Window stars (Church+, 2011)
J/A+A/559/A10 :  Arp 102B spectral optical monitoring (Shapovalova+, 2013)
J/A+A/568/A90 :  Herschel/SPIRE spectra in Arp 299 (Rosenberg+, 2014)
J/A+A/587/A78 :  LARS VIII. Spatially resolved Halpha kinematics (Herenz+, 2016)
J/A+A/590/A25 :  Arp 220 HCN and HCO^+^ data cubes (Martin+, 2016)
J/A+A/593/A86 :  Arp 220 LOFAR radio images at 150MHz (Varenius+, 2

Alternatively you can also search for the table with `pyvo` using the Table Access Protocol (TAP) to query VizieR. In this case, you first need to define the TAP endpoint of VizieR and then send a query written in ADQL (an SQL-like query language optimised for astronomy) to this endpoint. Finally we again print out the result of this query.

In [5]:
tap_vizier = pyvo.dal.TAPService(
    'https://tapvizier.cds.unistra.fr/TAPVizieR/tap')
query = """SELECT  *  FROM tap_schema.tables 
               WHERE table_name LIKE '%arp%' """
catalog_list_arp = tap_vizier.search(query).to_table()
catalog_list_arp['table_name', 'description']


table_name,description
object,object
J/ApJS/176/374/warps,"WARPS-II catalog, with additional clusters and candidates ( Horner D.J., Perlman E.S., Ebeling H., Jones L.R., Scharf C.A., Wegner G., Malkan M., Maughan B.)"
J/A+A/664/A53/harps15,"RV data taken with HARPS spectrograph ( Stalport M., Delisle J.-B., Udry S., Matthews E., Bourrier V., Leleu A.)"
J/A+A/614/A133/harps_b,"HARPS measurements of HD215152 taken after the fiber upgrade ( Delisle J.-B., Segransan D., Dumusque X., et al.)"
J/MNRAS/511/1043/harpsccf,"HARPS CCF SCALPELS data ( Wilson T.G., Goffo E., Alibert Y., et al.)"
J/A+A/614/A133/harps_a,"HARPS measurements of HD215152 taken before the fiber upgrade ( Delisle J.-B., Segransan D., Dumusque X., et al.)"
J/A+A/629/A111/harps,"HARPS time-series ( Cloutier R., Astudillo-Defru N., Bonfils X., et al.)"
J/A+A/666/A143/harps,"HARPS-S radial velocities of AD Leo (table A4) ( Kossakowski D., Kuerster M., Henning T., et al.)"
VII/192/arpord,list of Arp views with imaging data ( Webb D.)
J/A+A/665/A8/arp2,"Extra-tidal stars belonging to Arp 2 ( Kundu R., Navarrete C., Sbordone L., Carballo-Bello J.A., Fernandez-Trincado J.G., Minniti D., Singh H.P.)"


The catalogue that we are interested in today is "VII/192" and its description is "Arp's Peculiar Galaxies (Webb 1996)". In the TAP query we already see that there are two tables in this catalogue: `VII/192/arplist` and `VII/192/arpord`. To be able to work with these data, let's load them into this notebook. 

Again we can use either `astroquery` or `pyvo`. Which one you chose is up to you and which one you like better. Attention **Spoiler:** over the week you'll learn more about TAP and ADQL.

In oder to get the full catalogue with `astroquery`, we first set the row limit to infinite (i.e. `-1` in the notation of this package) and then ask `Vizier` to write the content of the catalogue into the variable `catalogs_arp` (second line):

In [6]:
Vizier.ROW_LIMIT = -1
catalogs_arp = Vizier.get_catalogs('VII/192')


Now let's inspect, what we got:

In [7]:
print(catalogs_arp)


TableList with 2 tables:
	'0:VII/192/arpord' with 12 column(s) and 338 row(s) 
	'1:VII/192/arplist' with 13 column(s) and 592 row(s) 


As we have seen in the first part of the tutorial, the catalogue "Arp's Peculiar Galaxies (Webb 1996)" comes with two tables: arpord and arplist. As you can see from the printout `Vizier` has downloaded both of them. However, we are still only interested in the arplist (index `1` in the TableList). Therefore, we make Python write only the arplist table into a new variable `table_arplist` (first line) and then display the table (second line):

In [8]:
table_arplist = catalogs_arp[1]
table_arplist


Arp,Name,VT,u_VT,dim1,dim2,u_dim2,MType,Uchart,RAJ2000,DEJ2000,Simbad,NED
,,mag,,arcmin,arcmin,,,,'h:m:s','d:m:s',,
int16,str16,float32,str1,float32,float32,str1,str14,int16,str10,str9,str6,str3
249,UGC 12891,16.2,,--,--,,,--,00 00 19.3,+22 59 26,Simbad,NED
249,UGC 12891,16.2,,1.3,0.5,,,--,00 00 21.6,+22 59 42,Simbad,NED
112,NGC 7805,13.3,,1.2,0.9,,SAB0^0: pec,89,00 01 26.9,+31 26 02,Simbad,NED
112,NGC 7806,13.5,,1.1,0.8,,SA(rs)bc? pec,89,00 01 30.2,+31 26 33,Simbad,NED
130,IC 5378,15.6,,0.5,--,,SBc,--,00 02 37.7,+16 39 08,Simbad,NED
130,IC 5378,15.3,,--,--,,E,--,00 02 37.8,+16 38 37,Simbad,NED
51,MGC-02-01-24,15.0,,0.8,--,,,--,00 06 16.8,-13 26 53,Simbad,NED
144,NGC 7828,14.4,,0.9,0.5,,Ring A,260,00 06 27.1,-13 24 58,Simbad,NED


Nice, we got the data we want. Before we move on, let's have a look at how we can use `pyVO` to get the same table. Just for safety we define again the VizieR TAP endpoint. It is the same one as above, but if the cells of this notebook are not run one after another, this safety measure ensure we always query the right service ;) 

The we write a simple query telling VizieR that we want all the rows and all the columns from the table `VII/192/arplist`. Note that we have to encompass the table name with quotation marks due to the special character `/` in the table name. 

In [9]:
query = """SELECT  *  FROM "VII/192/arplist" """
table_arplist = tap_vizier.search(query).to_table()
table_arplist


recno,Arp,Name,VT,u_VT,dim1,dim2,u_dim2,MType,Uchart,RAJ2000,DEJ2000
,,,mag,,arcmin,arcmin,,,,deg,deg
int32,int16,object,float64,str1,float64,float64,str1,object,int16,float64,float64
112,333,NGC 1024,12.1,,3.9,1.4,,(R')SA(r)ab,175,39.80083333333333,10.847222222222221
120,200,NGC 1134,12.1,,2.5,0.9,,S?,175,43.42124999999999,13.015277777777776
119,190,UGC 02320,15.2,,0.5,0.3,,Multiple Sys,--,42.58291666666666,12.889444444444443
89,290,IC 0195,14.3,,1.6,0.8,,S0,--,30.935833333333328,14.708611111111109
90,290,IC 0196,14.2,,2.8,1.4,,S0-,--,30.95833333333333,14.73972222222222
111,258,UGC 02140A,15.5,,0.9,0.2,,SB.0*/,--,39.78874999999999,18.367499999999996
110,258,UGC 02140,15.4,,1.7,0.7,,IBS9P,--,39.77583333333333,18.382777777777775
109,258,Hickson 18C,16.1,,--,--,,S?,--,39.774583333333325,18.388333333333332


There we go, we now know two nifty ways to get VizieR tables with Python. Obviously there are many other things you can do with this tools. For example if you set your TAP endpoint to `http://simbad.u-strasbg.fr:80/simbad/sim-tap` instead of the VizieR enpoint, you can query SIMBAD just like that. 

Note that not all rows but only the first and last 10 rows for all columns are shown in the display. To finish off this little excursion, we now want to visualise the location of the entries of this table in our Aladin Lite widget. To do so, we tell Python to take the variable `aladin` (remember that this was the name of the widget) and add the table `table_arplist` to it:

In [10]:
aladin.add_table(table_arplist)


/home/manon.marchand/.local/lib/python3.8/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Now scroll back up to the Aladin Lite widget. You will find that the location of the sources in the arplist table are marked with coloured symbols. Again you can zoom in and out to look at the different sources. If you click on one of the colour symbols, you will be able to see the corresponding row in the bottom of the Aladin Lite widget. 